In [25]:
import pandas as pd 
import json 
import random

In [28]:
# Few-shot 샘플 생성 위한 코드

# 타겟 데이터셋
fname = 'setting2_2.json'
fdata = json.load(open(f'{fname}'))

# 샘플 데이터셋 생성
foldername = 'ReClor_4shot'
import os 
if not os.path.exists(foldername):
    os.makedirs(foldername)

# Option Question과 MCQA Question을 가르는 기준
def answer_if_yes_or_no(answer):
    if 'yes' in answer or 'no' in answer:
        return True
    else:
        return False

# Train 데이터로부터 Option Question과 MCQA Question 추출
tdata_MCQA = [x for x in fdata['train'] if not answer_if_yes_or_no(x['answer'])]
tdata_option = [x for x in fdata['train'] if answer_if_yes_or_no(x['answer'])]

In [29]:
# ReClor에 있는 샘플 4개씩 샘플 (R 4개 O 4개)
random.seed(42)
tdata_MCQA_sample = random.sample(tdata_MCQA, 4)
random.seed(44)
tdata_option_sample = random.sample(tdata_option, 4)

# tdata_MCQA_sample의 ['answer'] 값 개수 분포 확인
import pandas as pd
mcqa_sample_df = pd.DataFrame(tdata_MCQA_sample)

print("MCQA", mcqa_sample_df['answer'].value_counts())

# tdata_option_sample의 ['answer'] 값 개수 분포 확인
option_sample_df = pd.DataFrame(tdata_option_sample)
print("Option",option_sample_df['answer'].value_counts())

mcqa_fname = f'{foldername}/MCQA_fewshot.json'
option_fname = f'{foldername}/Option_fewshot.json'

# 샘플 데이터셋 저장
with open(mcqa_fname, 'w') as f:
    json.dump(tdata_MCQA_sample, f, indent=4)

with open(option_fname, 'w') as f:
    json.dump(tdata_option_sample, f, indent=4)

MCQA answer
(B)    2
(A)    1
(D)    1
Name: count, dtype: int64
Option answer
no.     2
yes.    2
Name: count, dtype: int64


In [12]:
output_folder = ''
# tdata_MCQA_sample json 저장
fname_output = f'{output_folder}setting2_EM_test_MCQA.json'
with open(fname_output, 'w') as f:
    json.dump(tdata_MCQA_sample, f, indent=4)

# tdata_option_sample json 저장
fname_output = f'{output_folder}setting2_EM_test_option.json'
with open(fname_output, 'w') as f:
    json.dump(tdata_option_sample, f, indent=4)
    

NameError: name 'tdata_MCQA_sample' is not defined

In [20]:
def convert_llama2_prompt_format(system_prompt, user_prompt, delimiter="", system_response=None):
    system_prompt = system_prompt.strip()
    user_prompt = user_prompt.strip()
    if system_response is not None:
        system_response = system_response.strip()
    assert not any([tag in system_prompt for tag in SPECIAL_TAGS])
    assert not any([tag in user_prompt for tag in SPECIAL_TAGS])
    prompt = f"{B_INST} {(B_SYS + system_prompt + E_SYS + user_prompt).strip()} {delimiter} {E_INST}"
    if system_response is None:
        return prompt
    else:
        return prompt + ' ' + system_response
    
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

SPECIAL_TAGS = [B_INST, E_INST, "<<SYS>>", "<</SYS>>"]
UNSAFE_ERROR = "Error: special tags are not allowed as part of the prompt."

system_prompt = f'I will ask you a question. Answer to the question.'
options = ""
# return input/input&output
delimiter = f"{options}"
user_prompt = tdata_MCQA[0]['question']
input_string = convert_llama2_prompt_format(system_prompt, user_prompt, delimiter=delimiter)
input_string

"[INST] <<SYS>>\nI will ask you a question. Answer to the question.\n<</SYS>>\n\n<Passage>: In rheumatoid arthritis, the body' s immune system misfunctions by attacking healthy cells in the joints causing the release of a hormone that in turn causes pain and swelling. This hormone is normally activated only in reaction to injury or infection. A new arthritis medication will contain a protein that inhibits the functioning of the hormone that causes pain and swelling in the joints.\n<Question>: The statements above, if true, most strongly support which one of the following conclusions?\nChoose the correct answer from the following options.\n・ Unlike aspirin and other medications that reduce pain and swelling and that are currently available, the new medication would repair existing cell damage that had been caused by rheumatoid arthritis.\n・ A patient treated with the new medication for rheumatoid arthritis could sustain a joint injury without becoming aware of it.\n・ Joint diseases othe

In [7]:
import json 

# ReClor에 있는 샘플 4개씩 사용함 (R 4개 O 4개)
train_fname = 'label_train/setting3_EM.json'
train_fdata = json.load(open(f'{train_fname}'))

In [10]:
r_count = 0
o_count = 0

for row in train_fdata['train']:
    if 'option' in row['qid']:
        o_count += 1
    else:
        r_count += 1

print(f'RCQA: {r_count}, Option: {o_count}')


RCQA: 3695, Option: 3696
